https://www.youtube.com/watch?v=QSR0RCC3Mes


https://github.com/SchoolOfAISaoPaulo/aulas/blob/master/19_Aula_2019/2019_Aula19_LSTM_VictorVenites.ipynb

In [248]:
#https://analyticsindiamag.com/how-to-do-multivariate-time-series-forecasting-using-lstm/ #scaler separado
#https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/ #scaler junto
#nao vi https://www.youtube.com/watch?v=iP4323wII-I

Pacetes

In [249]:
#importar pacotes

#tratamento de dados
import pandas as pd
import numpy as np

#Modelagem
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

#Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

#dados aleatorios
import random



In [250]:
# Defina as sementes aleatórias para reprodutibilidade
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)

window_size = 10
features = 4

Dados

In [251]:
dados_finais = pd.read_csv('C:/Users/eduar/OneDrive/Área de Trabalho/Python/bitcoin/gold/bases/dados_finais.csv',parse_dates=True)

print(dados_finais.count())

Date             3345
btc              3345
sp500            3345
ouro             3345
interest_rate    3345
dtype: int64


In [252]:
dados_finais.tail()

,Date,btc,sp500,ouro,interest_rate
3340,2023-11-09,36693.125000,4347.350098,1964.199951,5.263
3341,2023-11-10,37313.968750,4415.240234,1932.599976,5.250
3342,2023-11-11,37138.050781,4415.240234,1932.599976,5.250
3343,2023-11-12,37054.519531,4415.240234,1932.599976,5.250
3344,2023-11-13,36502.355469,4411.549805,1945.500000,5.260


Pré-processamento

In [253]:
scaler = MinMaxScaler(feature_range=(-1,1))
scalery = MinMaxScaler(feature_range=(-1,1))

data2 = dados_finais[["btc","sp500","ouro","interest_rate"]] #multivariado
#data2 = dados_finais[["btc"]] #univariado

scaled_data2 = scaler.fit_transform(data2.values)

data2y = dados_finais[["btc"]] #multivariado
scaled_data2y = scalery.fit_transform(data2y.values) #multivariado

def create2XY(data,window_size):
    x = []
    y = []
    
    for i in range(len(data)-window_size):
        row = data[i:i+window_size]
        x.append(row)
        label = data[i+window_size]
        y.append(label)
    return np.array(x),np.array(y)

#x2,y2 = create2XY(scaled_data2,window_size) #univariado

In [254]:
def create_multivariate_X(data, window_size=10):
    x = []
    for i in range(len(data) - window_size):
        row = data[i:i + window_size]
        x.append(row)
        
    return np.array(x)

x2 = create_multivariate_X(scaled_data2,window_size=window_size)

def create_multivariate_Y(data, target_column=0, window_size=10):
    y = []
    for i in range(len(data) - window_size):
        label = data[i + window_size, target_column]
        y.append(label)

    return np.array(y)

y2 = create_multivariate_Y(scaled_data2y,0,window_size=window_size)

Modelagem

In [255]:
x_train,y_train = x2[:2900],y2[:2900]
x_val,y_val = x2[2900:3000],y2[2900:3000]
x_test,y_test = x2[3000:],y2[3000:]

In [256]:
es = EarlyStopping(monitor="val_loss",patience=2)
model = Sequential()
model.add(InputLayer((window_size,features)))
model.add(LSTM(4))
model.add(Dense(1,"linear"))
model.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01))
model.fit(x_train,y_train,validation_data = (x_val,y_val),epochs=10,callbacks=[es], batch_size = 1) 


Epoch 1/10
2900/2900 [==============================] - 13s 4ms/step - loss: 0.0039 - val_loss: 0.0014
Epoch 2/10
2900/2900 [==============================] - 10s 3ms/step - loss: 0.0014 - val_loss: 0.0021
Epoch 3/10
2900/2900 [==============================] - 11s 4ms/step - loss: 0.0014 - val_loss: 4.5074e-04
Epoch 4/10
2900/2900 [==============================] - 11s 4ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 5/10
2900/2900 [==============================] - 11s 4ms/step - loss: 0.0011 - val_loss: 3.3575e-04
Epoch 6/10
2900/2900 [==============================] - 12s 4ms/step - loss: 0.0010 - val_loss: 3.5920e-04
Epoch 7/10
2900/2900 [==============================] - 12s 4ms/step - loss: 0.0011 - val_loss: 3.4942e-04


In [257]:
# Gerar previsões no conjunto de teste
y_pred = model.predict(x_test)
y_pred = scalery.inverse_transform(y_pred.reshape(-1, 1))
y_pred = y_pred.flatten()

y_test = scalery.inverse_transform(y_test.reshape(-1, 1))
y_test = y_test.flatten()


# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
loss = model.evaluate(x_test, y_test)


print("Mean Absolute Error (MAE):", mae)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")
print(f"Loss: {loss}")

11/11 [==============================] - 1s 3ms/step - loss: 722701824.0000
Mean Absolute Error (MAE): 465.2865146921642
Mean Squared Error (MSE): 475112.3016610787
R-squared (R2): 0.9751644130846513
Loss: 722701824.0


In [258]:
y_test.shape

(335,)

In [259]:
y_pred.shape

(335,)

- Univariavel e Multivariavel
- Já definido hipers
- uni > multi. Verificar se o multi sem alguma feature fica melhor que só bit